In [1]:
import os

import numpy as np
from sklearn.decomposition import PCA

from utils import load_hidden_representations_from_hdf5, read_templates_from_file

----

In [2]:
# params
log_dir = "/logfiles"
model = "bigscience-T0"
module = "encoder" # encoder
task = "rte"
# task = "cb"
# task = "wic"

In [3]:
assert module == "encoder"

## Prompts

In [4]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_prompt.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_target_yes_no.csv")
display(df)

,name,template,category,includes_targets,targets,target_ids,shuffle
0,gpt_3_yes_no_with_targets,{premise} Question: {hypothesis} Yes or No?,instructive,True,"▁Yes, ▁No","0, 1",False
1,gpt_3_true_false_with_targets,{premise} Question: {hypothesis} True or False?,instructive,True,"▁True, ▁Fal","0, 1",False
2,gpt_3_cat_dog_with_targets,{premise} Question: {hypothesis} Cat or Dog?,instructive,True,"▁Cat, ▁Dog","0, 1",False
3,gpt_3_yes_no_without_targets,{premise} Question: {hypothesis}?,instructive,False,"▁Yes, ▁No","0, 1",False


## PCA

In [5]:
def unison_shuffled_copies(a, b):
    # from: https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [6]:
if task == 'rte':
    use_pattern = [
        "gpt_3_yes_no_with_targets",
        "gpt_3_true_false_with_targets",
        "gpt_3_cat_dog_with_targets",
        "gpt_3_yes_no_without_targets",
    ]
elif task == 'cb':
    use_pattern = [
        "null_pattern",
        "null_pattern_reversed",
        "gpt_3_true_false_neither",
        "gpt_3_yes_no_maybe",
        "mnli_crowdsource",
        "always_sometimes_never",
        "based_on_previous_passage",
        "infer",
        "claim",
        "consider",
        "follow",
        "imply",
        "guaranteed",
        "guaranteed_possible",
        "justified",
        "must_be_true",
        "should_assume",
        "take_the_following",
    ]
elif task == 'wic':
    use_pattern = [
        "gpt_3",
        "gpt_3_yes_no",
        "affirmation",
        "grammar_homework",
        "polysemous",
        "question_context",
        "question_meaning",
        "question_meaning_yes_no",
        "same_sense",
        "similar_sense",
        "similar_sense_yes_no",
    ]

In [7]:
# for layer in range(0, 10):
for layer in range(0, 25):
# for layer in range(24, 25):
    print('layer=', layer)
    file_names, prompt_names = [], []

    for _, row in df.iterrows():
        if row['name'] in use_pattern:
            file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_layer{layer}_avg.hdf5",)
            prompt_names.append(row['name'])


    # load hidden representations from hdf5 file
    representations = None
    classes = []
    n_sequences = 0

    for idx, file_name in enumerate(file_names):
        hidden_representations = load_hidden_representations_from_hdf5(os.path.join(log_dir, file_name))
        # print(hidden_representations.shape)
        n_sequences = hidden_representations.shape[0]

        if representations is None:
            representations = hidden_representations
        else:
            representations = np.concatenate((representations, hidden_representations), axis=0)

        classes += n_sequences * [idx] # assign representations to classes
    
    classes = np.asarray(classes)

    # shuffle representations and classes
    X, y = unison_shuffled_copies(representations, classes)
    print(X.shape, y.shape)

    # perform PCA on hidden representations
    print('PCA for prompts:', prompt_names)

    for n_components in range(1, 4):
        pca = PCA(n_components=n_components)
        pca.fit(X)

        # variance explained by each of the principal components
        print(f"model:{model}; module:{module}; layer:{layer}; n_components: {n_components}; variance explained: {pca.explained_variance_ratio_}")
        print(np.sum(pca.explained_variance_ratio_))
    print('\n')


layer= 0


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5695.07it/s]


(1108, 4096) (1108,)
PCA for prompts: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets', 'gpt_3_cat_dog_with_targets', 'gpt_3_yes_no_without_targets']
model:bigscience-T0; module:encoder; layer:0; n_components: 1; variance explained: [0.9520281]
0.9520281
model:bigscience-T0; module:encoder; layer:0; n_components: 2; variance explained: [0.9520281  0.00233076]
0.9543589
model:bigscience-T0; module:encoder; layer:0; n_components: 3; variance explained: [0.95202833 0.00233076 0.00181649]
0.9561756


layer= 1


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5511.94it/s]


(1108, 4096) (1108,)
PCA for prompts: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets', 'gpt_3_cat_dog_with_targets', 'gpt_3_yes_no_without_targets']
model:bigscience-T0; module:encoder; layer:1; n_components: 1; variance explained: [0.09682094]
0.09682094
model:bigscience-T0; module:encoder; layer:1; n_components: 2; variance explained: [0.09682087 0.06569661]
0.16251749
model:bigscience-T0; module:encoder; layer:1; n_components: 3; variance explained: [0.09682088 0.06569663 0.04756508]
0.2100826


layer= 2


...

(1108, 4096) (1108,)
PCA for prompts: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets', 'gpt_3_cat_dog_with_targets', 'gpt_3_yes_no_without_targets']
model:bigscience-T0; module:encoder; layer:22; n_components: 1; variance explained: [0.18796729]
0.18796729
model:bigscience-T0; module:encoder; layer:22; n_components: 2; variance explained: [0.18796754 0.12157819]
0.30954573
model:bigscience-T0; module:encoder; layer:22; n_components: 3; variance explained: [0.1879673  0.12157818 0.06081048]
0.37035596


layer= 23


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5845.82it/s]


(1108, 4096) (1108,)
PCA for prompts: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets', 'gpt_3_cat_dog_with_targets', 'gpt_3_yes_no_without_targets']
model:bigscience-T0; module:encoder; layer:23; n_components: 1; variance explained: [0.18853156]
0.18853156
model:bigscience-T0; module:encoder; layer:23; n_components: 2; variance explained: [0.18853177 0.10610338]
0.29463515
model:bigscience-T0; module:encoder; layer:23; n_components: 3; variance explained: [0.1885317  0.10610352 0.06170622]
0.35634142


layer= 24


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5434.92it/s]


(1108, 4096) (1108,)
PCA for prompts: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets', 'gpt_3_cat_dog_with_targets', 'gpt_3_yes_no_without_targets']
model:bigscience-T0; module:encoder; layer:24; n_components: 1; variance explained: [0.12169816]
0.12169816
model:bigscience-T0; module:encoder; layer:24; n_components: 2; variance explained: [0.12169814 0.05561725]
0.17731538
model:bigscience-T0; module:encoder; layer:24; n_components: 3; variance explained: [0.12169813 0.05561724 0.04918166]
0.22649701


